In [1]:
'''GENERAL'''
import os
import collections
import glob
'''DATA'''
import numpy as np
import pandas as pd
import databricks.koalas as ks
'''SIGNATURE'''
!fortune | cowsay -f dragon

 _____________________________________ 
/ When you are in it up to your ears, \
\ keep your mouth shut.               /
 ------------------------------------- 
      \                    / \  //\
       \    |\___/|      /   \//  \\
            /0  0  \__  /    //  | \ \    
           /     /  \/_/    //   |  \  \  
           @_^_@'/   \/_   //    |   \   \ 
           //_^_/     \/_ //     |    \    \
        ( //) |        \///      |     \     \
      ( / /) _|_ /   )  //       |      \     _\
    ( // /) '/,_ _ _/  ( ; -.    |    _ _\.-~        .-~~~^-.
  (( / / )) ,-{        _      `-.|.-~-.           .~         `.
 (( // / ))  '/\      /                 ~-. _ .-~      .-~^-.  \
 (( /// ))      `.   {            }                   /      \  \
  (( / ))     .----~-.\        \-'                 .~         \  `. \^-.
             ///.----..>        \             _ -~             `.  ^-`  ^-_
               ///-._ _ _ _ _ _ _}^ - - - - ~                     ~-- ,.-~
               

# Initial Processing

I have a directory of csvs from stat.ink which contains player submitted match data. I want to process them into one cleaned database. I will first clean one csv to devolop a pipeline.

In [2]:
# Gather list of raw data filenames
raw = glob.glob(os.path.join('data/raw', '*.csv'))
# Count the number of csvs
len(raw)

684

My data is currently spread among hundreds of csvs.

In [3]:
raw[:5]

['data/raw/2018-09-19.csv',
 'data/raw/2019-06-03.csv',
 'data/raw/2018-07-20.csv',
 'data/raw/2019-06-17.csv',
 'data/raw/2018-07-08.csv']

In [4]:
# Load one to inspect
test = pd.read_csv(raw[0])
test.head()

,# period,game-ver,lobby-mode,lobby,mode,stage,time,win,knockout,A1-weapon,...,B3-level,B4-weapon,B4-kill-assist,B4-kill,B4-assist,B4-death,B4-special,B4-inked,B4-rank,B4-level
0,2018-08-31T12:00:00+00:00,3.2.2,gachi,standard,hoko,sumeshi,300,bravo,False,dynamo,...,19,maneuver,10.0,9.0,1.0,5.0,4.0,1298.0,b+,23.0
1,2018-08-31T12:00:00+00:00,3.2.2,gachi,standard,hoko,sumeshi,167,alpha,True,dynamo,...,45,furo,2.0,2.0,0.0,6.0,2.0,716.0,b+,45.0
2,2018-08-31T12:00:00+00:00,3.2.2,gachi,standard,hoko,sumeshi,108,alpha,True,dynamo,...,13,longblaster,4.0,4.0,0.0,2.0,2.0,623.0,b,27.0
3,2018-09-06T12:00:00+00:00,3.2.2,gachi,standard,hoko,shottsuru,312,bravo,False,dynamo,...,15,maneuver,3.0,1.0,2.0,7.0,1.0,0.0,b+,24.0
4,2018-09-06T12:00:00+00:00,3.2.2,gachi,standard,hoko,zatou,66,bravo,True,dynamo,...,31,furo,5.0,3.0,2.0,2.0,1.0,354.0,b+,26.0


### Preliminary Feature Selection

Since this data is from match results and I want to create a predictive model for win rate, there are many features that cannot be used as they are only knowable at the end of a match:

- period: when the match was played
    - irrelevant because this is not a timeseries model
    - version number better covers patch changes
- time: how long the match took
- knockout: if the match was won through the objective as opposed to a time-out
- player kill/assist/death/special/inked: player statistics calculated at the end of each match

In [5]:
def splatoon_drop(df):
    """ 
    Drops unnessecary features in stat.ink dataframes
  
    Parameters: 
    df (DataFrame): pandas or koalas dataframe on which to drop values
  
    Returns: 
    DataFrame: the dataframe with dropped features
  
    """
    # initialize list with first features
    drop_lst = ['# period','time', 'knockout']
    # concatanate player statistics features for each player
    for player in ['A1', 'A2', 'A3', 'A4', 'B1', 'B2', 'B3', 'B4']:
        drop_lst += [player+'-kill-assist', player+'-kill', player+'-assist',
                     player+'-death', player+'-special', player+'-inked']
    # drop features in list
    return df.drop(drop_lst, axis=1)

In [6]:
test = splatoon_drop(test)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2445 entries, 0 to 2444
Data columns (total 30 columns):
game-ver      2445 non-null object
lobby-mode    2445 non-null object
lobby         2445 non-null object
mode          2445 non-null object
stage         2445 non-null object
win           2445 non-null object
A1-weapon     2443 non-null object
A1-rank       1607 non-null object
A1-level      2445 non-null int64
A2-weapon     2442 non-null object
A2-rank       1606 non-null object
A2-level      2445 non-null int64
A3-weapon     2442 non-null object
A3-rank       1606 non-null object
A3-level      2445 non-null int64
A4-weapon     2438 non-null object
A4-rank       1604 non-null object
A4-level      2443 non-null float64
B1-weapon     2439 non-null object
B1-rank       1603 non-null object
B1-level      2445 non-null int64
B2-weapon     2439 non-null object
B2-rank       1604 non-null object
B2-level      2445 non-null int64
B3-weapon     2435 non-null object
B3-rank       1604 non

### Exploding the rows

Since I am interested in making predictions based off data from one player, I will explode each match data into 8 separate rows, one for every player. The function will drop data from A1 by default as A1 is the player who submitted the data, which can lead to bias. I have more than enough data to drop it.

In [7]:
def splatoon_explode(df, drop=True):
    """ 
    Explodes rows of stat.ink dataframes into one row for each player.
    This function is intended to be run after splatoon_drop and
    is hardcoded to accept its output features.
  
    Parameters: 
    df (DataFrame): Pandas dataframe on which to explode
  
    Returns: 
    koalas: Exploded dataframe stored in a koalas database
  
    """
    # get feature names
    features = df.columns.tolist()
    # features for all players
    shared = features[:5]
    # features for specific player
    a1 = features[6:9]
    a2 = features[9:12]
    a3 = features[12:15]
    a4 = features[15:18]
    b1 = features[18:21]
    b2 = features[21:24]
    b3 = features[24:27]
    b4 = features[27:]
    # group players by team
    if drop:
        a_team = [a2, a3, a4]
    else:
        a_team = [a1, a2, a3, a4]
    b_team = [b1, b2, b3, b4]

    # initialize temporaty storage list
    tmp = collections.deque()
    # iterate through rows
    for index, row in df.iterrows():
        # get features common to both teams
        both = [index] + row[shared].tolist()
        # append features for each a team player
        for player in a_team:
            tmp.append(both + row[player].tolist() + [row.win == 'alpha'])
        # append features for each b team player
        for player in b_team:
            tmp.append(both + row[player].tolist() + [row.win == 'bravo'])

    #create new column names, including the index of the match the data was from
    new_cols = ['match'] + shared + ['weapon', 'rank', 'level', 'win']
    # return new koalas database
    return pd.DataFrame(list(tmp), columns=new_cols)

In [8]:
test = splatoon_explode(test)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17115 entries, 0 to 17114
Data columns (total 10 columns):
match         17115 non-null int64
game-ver      17115 non-null object
lobby-mode    17115 non-null object
lobby         17115 non-null object
mode          17115 non-null object
stage         17115 non-null object
weapon        17074 non-null object
rank          11231 non-null object
level         17112 non-null float64
win           17115 non-null bool
dtypes: bool(1), float64(1), int64(1), object(7)
memory usage: 1.2+ MB


In [9]:
test.head(20)

,match,game-ver,lobby-mode,lobby,mode,stage,weapon,rank,level,win
0,0,3.2.2,gachi,standard,hoko,sumeshi,wakaba,b,14.0,False
1,0,3.2.2,gachi,standard,hoko,sumeshi,maneuver,b+,60.0,False
2,0,3.2.2,gachi,standard,hoko,sumeshi,liter4k,b+,21.0,False
3,0,3.2.2,gachi,standard,hoko,sumeshi,splatcharger,b+,37.0,True
4,0,3.2.2,gachi,standard,hoko,sumeshi,nzap89,b-,18.0,True
5,0,3.2.2,gachi,standard,hoko,sumeshi,jetsweeper,b-,19.0,True
6,0,3.2.2,gachi,standard,hoko,sumeshi,maneuver,b+,23.0,True
7,1,3.2.2,gachi,standard,hoko,sumeshi,clashblaster,b+,39.0,True
8,1,3.2.2,gachi,standard,hoko,sumeshi,carbon,b+,46.0,True
9,1,3.2.2,gachi,standard,hoko,sumeshi,splatscope,b-,18.0,True


The processing worked, time to save it as a pickle file.

In [12]:
# test.to_pickle('data/test.pkl')

### Completed Pipeline

I test out the pipeline here in the notebook but it is meant to be used called through the src/pipeline.py file. the pipeline is designed to read all csvs from data/raw and output them as converted pickles in data/dump. The paths are hardcoded in the program as I do not need it to change and would rather setup consistent folders than define the paths every time.

Here I test it out with the first two files.

In [29]:
in_path = 'data/raw/'
out_path = 'data/dump/'
l1 = len(in_path)

files = glob.glob(os.path.join(in_path, '*.csv'))

for file in files[:2]:
    Break
#     splatoon_explode(splatoon_drop(pd.read_csv(file))).to_pickle(out_path + file[l:-4] + '.pkl')

# Merging the daraframes

With conversion sucessful, I now have a directory full of processed pickle files that can be easily expanded as data is collected. I will merge them all into one large pickle file.

In [2]:
def splatoon_concat(path = 'data/dump'):
    """ 
    Concatanates multiple pickled dataframes from one directory into a dataframe
  
    Parameters: 
    path (string): optional specified file path
  
    Returns: 
    DataFrame: pandas dataframe containing data from all csvs.

    """
    # get filenames
    files = glob.glob(os.path.join(path, '*.pkl'))
    # concat all files
    return pd.concat((pd.read_pickle(f) for f in files))

In [4]:
df = splatoon_concat()
# df.to_pickle('data/by_player.pkl')
df.shape

(13145426, 10)

### Cleaning up the merge

I know there were a few missing values in the original csvs but I did not want to drop before exploding as a missing value for one player would caue the entire match to be dropped.

In [5]:
df = pd.read_pickle('data/by_player.pkl')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13145426 entries, 0 to 15623
Data columns (total 10 columns):
match         int64
game-ver      object
lobby-mode    object
lobby         object
mode          object
stage         object
weapon        object
rank          object
level         float64
win           bool
dtypes: bool(1), float64(1), int64(1), object(7)
memory usage: 1015.5+ MB


In [3]:
np.sum(df.isnull())

match               0
game-ver            0
lobby-mode          0
lobby               0
mode                0
stage               0
weapon          30283
rank          4594541
level            3740
win                 0
dtype: int64

##
Weapon is absolutely critical for my model and I have plenty of observations so I can safely drop them.

In [3]:
df = df[df.weapon.notnull()]
np.sum(df.isnull())

match               0
game-ver            0
lobby-mode          0
lobby               0
mode                0
stage               0
weapon              0
rank          4577927
level               0
win                 0
dtype: int64

In [4]:
df.shape

(12792037, 10)

In a stroke of luck, dropping null values for weapon also cleaned up level. I suspect missing rank to be from unranked (regular and fest) matches, so I will look into that.

In [5]:
df[df['rank'].isnull()]['lobby-mode'].unique()

array(['regular', 'gachi', 'fest'], dtype=object)

In [6]:
# gachi is the only ranked battle mode
np.sum(df[df['rank'].isnull()]['lobby-mode']=='gachi')

22540

It seems that there are some missing values in ranked battles, I will drop them.

In [5]:
df = df[(df['lobby-mode']!='gachi') | (df['rank'].notnull())]
df.shape

(12769497, 10)

Save to pickle to use in EDA and modeling.

In [ ]:
# df.to_pickle('data/clean.pkl')